In [ ]:
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from june import World 
from june.geography import Geography
from june.demography import Demography
from june.interaction import Interaction
from june.infection import Infection, InfectionSelector
from june.infection.health_index import Data2Rates
from june.infection.health_index.health_index import HealthIndexGenerator
from june.infection.transmission import TransmissionConstant
from june.groups import Hospitals, Schools, Companies, Households, CareHomes, Cemeteries, Universities
from june.groups.leisure import generate_leisure_for_config, Cinemas, Pubs, Groceries, Gyms
from june.groups.travel import *
from june.simulator import Simulator
from june.infection_seed import InfectionSeed
from june.policy import Policy, Policies
from june import paths
from june.hdf5_savers import load_geography_from_hdf5
from june.records import Record, RecordReader

from june.world import generate_world_from_geography
from june.hdf5_savers import generate_world_from_hdf5

# Initialize world

To initialize a certain world, we need to add the different components we want to have in it. First we specify what super areas (msoa) we want to create. We have included these ones, because they are known to contain hospitals, schools, care homes, and companies.

After creating the geography, we create the different components the worlds need to have such as care homes, companies ...

In [ ]:
CONFIG_PATH = paths.configs_path / "config_example.yaml"

In [ ]:
%%time 

geography = Geography.from_file(
    {
        "super_area": ["D07315", "D07339"]
    }
)

geography.hospitals = Hospitals.for_geography(geography)
geography.schools = Schools.for_geography(geography)
geography.companies = Companies.for_geography(geography)
geography.care_homes = CareHomes.for_geography(geography)
geography.universities = Universities.for_geography(geography)
world = generate_world_from_geography(geography, include_households=True)

## Commute, travel and leisure

In [ ]:
%%time

world.pubs = Pubs.for_geography(geography)
world.cinemas = Cinemas.for_geography(geography)
world.groceries = Groceries.for_geography(geography)
world.gyms = Gyms.for_geography(geography)
leisure = generate_leisure_for_config(world, config_filename=CONFIG_PATH)
leisure.distribute_social_venues_to_areas(
    areas=world.areas, super_areas=world.super_areas
)

In [ ]:
# initialise commuting travel
# Note that there are special configs for running on Rhineland-Palatinate only
travel = Travel(
    city_super_areas_filename=paths.data_path / "input/geography/cities_per_super_area_rlp.csv", 
    city_stations_filename=paths.configs_path / "defaults/travel/city_stations_RLP.yaml",
    commute_config_filename=paths.configs_path / "defaults/groups/travel/commute_RLP.yaml"
)
travel.initialise_commute(world)

We are also going to need some cemeteries...


In [ ]:
world.cemeteries = Cemeteries()

### If it took a long time to run the previous commands, it might be a good idea to save the world to reuse it later.

In [ ]:
world.to_hdf5("world.hdf5")

If we would like to load the world we saved, we just do

In [ ]:
world = generate_world_from_hdf5("world.hdf5")

In [ ]:
# and regenerate leisure in case we load it externally
leisure = generate_leisure_for_config(world, CONFIG_PATH)
# create travel as well
travel = Travel()

you have now a beautiful pre-pandemic world. 

# Adding the infection

The module in charge of infecting people is called the ``InfectionSelector``, which gives people a transmission time profile and a symptoms trajectory based on their age and sex (through the health index generator)

In [ ]:
selector = InfectionSelector.from_file()

# Adding the interaction

In [ ]:
interaction = Interaction.from_file(population=world.people)

Beta are the intensities of the interaction taking place at the different groups

In [ ]:
interaction.betas

Moreover, this interaction module uses contact matrices, that are different for different groups. These contact matrices shouldnt be modified for now. However they are a combination of conversational contact matrices, and physical contact matrices (see the BBC pandemic paper, from where these matrices are extracted https://www.medrxiv.org/content/10.1101/2020.02.16.20023754v2)

There is a parameter, ``alpha`` ($\alpha$), that combines these two matrices in the following way,


$\beta M \left(1 + (\alpha -1) \right) P$

where $\beta$ is the intensity of the interaction, and $P$ the physical contact matrix. A larger $\alpha$ produces more physical contacts. It is an overall number, non dependent of the particular group.


In [ ]:
interaction.alpha_physical

# Seed the disease

There are two options implemented in the seed at the moment, either you specify the number of cases and these are then homogeneously distributed by population to the different areas, or you use UK data on cases per region. For now use the first case.

In [ ]:
infection_seed = InfectionSeed(
    world, selector,
)

In [ ]:
n_cases = 50
infection_seed.unleash_virus(
    population=world.people,
    n_cases=n_cases) # play around with the initial number of cases

# Set policies

In [ ]:
policies = Policies.from_file()

We can have a look at one of the policies

In [ ]:
print(policies.individual_policies[8].__dict__)

# Run the simulation

The first thing we need is a place where to save our simulation results. For that we can use the record class,
and pass it to the simulator.

In [ ]:
record = Record(    
    record_path = 'results',    
    record_static_data=True,
) 

In [ ]:
record.static_data(world=world)

The simulator is the main module in charge of running the simulation. It coordinates the ``ActivityManager`` which is responsible of allocating people to the right groups given the current timestep, it updates the health status of the population, and it runs the interaction over the different groups. All of these modules can be modified by policies at any given time.

Since the timer configuration is a bit cumbersome, it is read from the config file at https://github.com/IDAS-Durham/JUNE/blob/master/june/configs/config_example.yaml

In [ ]:
simulator = Simulator.from_file(
    world=world,
    infection_selector=selector,
    interaction=interaction, 
    config_filename = CONFIG_PATH,
    leisure = leisure,
    travel = travel,
    record=record,
    policies = policies
)

In [ ]:
%%time
simulator.run()

While the simulation runs (and afterwards) we can launch the visualization webpage by running
```python june/visualizer.py path/to/results``` 

# Getting the results

The record saves a lot of information about the simulation, and it can be a bit overwhelming to look at everyting.
In the results folder (if you didn't change the path), we have a few extra contents:


In [ ]:
!ls results

Checkpoints would allow us to resume the simulation later, and the config and policies are there to remember what you used to run the code.
The ``summary.csv`` is useful to have a first glance at results:

In [ ]:
summary = pd.read_csv("results/summary.csv", index_col=0)
summary.head()

In [ ]:
summary.plot(y="daily_infected")

If we want to get the full details, we can read the record:

In [ ]:
read = RecordReader("./results")

In [ ]:
read.regional_summary.head(3) # this is the equivalent of the summary.csv

# Asking questions to the records

## Useful infections and death tables

In [ ]:
infections_df = read.get_table_with_extras("infections", "infected_ids")

In [ ]:
infections_df.head(10)

In [ ]:
deaths_df = read.get_table_with_extras("deaths", "dead_person_ids")

In [ ]:
deaths_df.head(10)

## Sero-prevalence by age

In [ ]:
age_bins = (0, 20, 60, 100)

In [ ]:
infected_by_age = infections_df.groupby(
    [pd.cut(infections_df["age"], bins=age_bins), "timestamp"]
).size()

In [ ]:
people_df = read.table_to_df("population")

In [ ]:
n_by_age = people_df.groupby(pd.cut(people_df["age"], bins=age_bins)).size()

In [ ]:
(100 * infected_by_age / n_by_age).xs(10).cumsum().plot(label="0,20")
(100 * infected_by_age / n_by_age).xs(30).cumsum().plot(label="20,60")
(100 * infected_by_age / n_by_age).xs(70).cumsum().plot(label="60,100")
plt.ylabel("Seroprevalence")
plt.legend()

## Care home deaths in hospital

In [ ]:
care_home_deaths_hospital = deaths_df[
    (deaths_df["location_specs"] == "hospital")
    & (deaths_df["residence_type"] == "care_home")
]
care_home_deaths_hospital = care_home_deaths_hospital.groupby(
    ["name_region", "timestamp"]
).size()

In [ ]:
if len(care_home_deaths_hospital) == 0:
    print("No care home deaths in the simulation")
else:
    care_home_deaths_hospital.unstack(level=0).plot()

## Where people get infected as a function of time

In [ ]:
locations_df = infections_df.groupby(["location_specs", "timestamp"]).size()
locations_df_pivot = (
    infections_df.groupby(["location_specs", "timestamp"])
    .size()
    .reset_index()
    .pivot(index="timestamp", columns="location_specs", values=0)
    .fillna(0)
    .groupby(pd.Grouper(freq="7D"))
    .sum()
)
locations_df_pivot = (
    locations_df_pivot.div(locations_df_pivot.sum(axis=1), axis=0) * 100
)

In [ ]:
fig, ax = plt.subplots()
bottom = None
weeks = list(range(1, len(locations_df_pivot) + 1))
for col in locations_df_pivot.columns:
    if bottom is None:
        ax.bar(weeks, locations_df_pivot[col], label=col)
        bottom = np.array(locations_df_pivot[col].values)
    else:
        ax.bar(weeks, locations_df_pivot[col], bottom=bottom, label=col)
        bottom += locations_df_pivot[col].values
ax.set_xlabel("Time [week]")
ax.set_ylabel("Percentage")
ax.legend(bbox_to_anchor=(1.05, 0.5), loc="center left", borderaxespad=0.0)

In [ ]:
import matplotlib.ticker as mtick

location_counts_df = locations_df.groupby("location_specs").size()
location_counts_df = 100 * location_counts_df / location_counts_df.sum()
ax = location_counts_df.sort_values().plot.bar()
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.ylabel("Percentage of infections at location")
plt.xlabel("location")

## Where people of certain age get infected as a function of time

In [ ]:
old_locations_df = (
    infections_df[infections_df.age > 65]
    .groupby(["location_specs", "timestamp"])
    .size()
)
old_locations_df = (
    infections_df[infections_df.age > 65]
    .groupby(["location_specs", "timestamp"])
    .size()
)
old_locations_df_pivot = (
    old_locations_df.groupby(["location_specs", "timestamp"])
    .size()
    .reset_index()
    .pivot(index="timestamp", columns="location_specs", values=0)
    .fillna(0)
    .groupby(pd.Grouper(freq="7D"))
    .sum()
)
old_locations_df_pivot = (
    old_locations_df_pivot.div(old_locations_df_pivot.sum(axis=1), axis=0) * 100
)

fig, ax = plt.subplots()
bottom = None
weeks = list(range(1, len(old_locations_df_pivot) + 1))
for col in old_locations_df_pivot.columns:
    if bottom is None:
        ax.bar(weeks, old_locations_df_pivot[col], label=col)
        bottom = np.array(old_locations_df_pivot[col].values)
    else:
        ax.bar(weeks, old_locations_df_pivot[col], bottom=bottom, label=col)
        bottom += old_locations_df_pivot[col].values
ax.set_xlabel("Time [week]")
ax.set_ylabel("Percentage")
ax.legend(bbox_to_anchor=(1.05, 0.5), loc="center left", borderaxespad=0.0)